In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
#상수
KW_LIBRARY = 'https://kupis.kw.ac.kr/'
WOS_QUERY = 'https://www-webofscience-com.libproxy.kw.ac.kr/wos/woscc/advanced-search'

QUARY = ('TS=(Federated Learning)' 
         ' AND '
         'WC=(Communication' ' OR '
         'Computer Science, Artificial Intelligence' ' OR '
         'Computer Science, Cybernetics' ' OR '
         'Computer Science, Hardware & Architecture' ' OR '
         'Computer Science, Information Systems' ' OR '
         'Computer Science, Interdisciplinary Applications' ' OR '
         'Computer Science, Software Engineering' ' OR '
         'Computer Science, Theory & Methods' ' OR '
         'Telecommunications)' )

EXPORT = '(overlay:export/exc)'

In [ ]:
#보면서 하게 왼쪽 반으로 맞춤
def set_driver():
    driver = webdriver.Chrome()
    driver.set_window_position(0, 0)
    driver.set_window_size(960, 1080)
    return driver
#중앙도서관 로그인 대기
def kw_lib_login(driver):
    driver.get(KW_LIBRARY)
    input('로그인 완료 후 아무 키 입력')
#새 탭 만들고 그리로 이동
def move_new_tab(driver, url):
    driver.execute_script(f"window.open('{url}');")
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(2.2)
#wos고급 검색으로 바로 이동후 쿼리입력(팝업 떠서 대기해야함)
def search_by_query(driver):
    move_new_tab(driver, WOS_QUERY)
    input('잡다한거 없앤 후 아무 키 입력')
    query_in = driver.find_element(By.XPATH, '//*[@id="advancedSearchInputArea"]')
    query_in.send_keys(QUARY)
    button = driver.find_element(By.XPATH, '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-search-home/div[2]/div[2]/app-input-route/app-search-advanced/app-advanced-search-form/form/div[3]/div[1]/div[1]/div/button[2]')
    button.click()
    time.sleep(6)
    doc_count = driver.find_element(By.XPATH, '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/app-search-friendly-display/div[1]/app-general-search-friendly-display/div/div/h1/span')
    value = int(doc_count.text.replace(',', ''))
    return value
#1000개 단위로만 받을 수 있어 문서 개수를 길이 천인 구간으로 분할
def make_interval(num):
    result = list(zip([i for i in range(1, num, 1000)], [i for i in range(1000, num, 1000)]))
    result.append((result[-1][1] + 1, result[-1][1] + num % 1000))
    return result
#한 구간에 대해 1000개 다운로드
def export_single_page(driver, start, end):
    driver.find_element(By.XPATH, '//*[@id="radio3"]/label/span[1]/span[2]').click()
    s = driver.find_element(By.XPATH, '//*[@id="mat-input-0"]')
    s.clear()
    s.send_keys(start)
    time.sleep(0.1)
    k = driver.find_element(By.XPATH, '//*[@id="mat-input-1"]')
    k.clear()
    k.send_keys(end)
    driver.find_element(By.XPATH, '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[1]/wos-select/button').click()
    time.sleep(0.1)
    driver.find_element(By.XPATH, '//*[@id="global-select"]/div/div/div[3]').click()
    time.sleep(0.1)
    driver.find_element(By.XPATH, '//*[@id="exportButton"]').click()
#전체 다운로드
def export(driver, interval):
    for i in interval:
        move_new_tab(driver, driver.current_url + EXPORT)
        export_single_page(driver, i[0], i[1])

In [ ]:
#쿼리에 대응하는 논문 전부 다운로드
def crawl():
    driver = set_driver()
    kw_lib_login(driver)
    doc_count = search_by_query(driver)
    export(driver, make_interval(doc_count))

crawl()